## Series Analysis

A notebook to analyze the results of Series, a parameterized series of experiments. See examples/run_series.

In [ ]:
import os
import pandas as pd
from IPython.display import display, HTML
from spikey.logging import Reader

In [ ]:
# Folder containing log files from a series of experiments
RESULTS_FOLDER = os.path.join('..', 'log', 'control')

In [ ]:
reader = Reader(folder=RESULTS_FOLDER)
reader.summary

In [ ]:
# TODO Not friendly with multi parameter series eg [('stdp_window', [40, 50, 60]), ('processing_time', [100, 200])]
experiment_params = reader.summary['results']['experiment_params']
if experiment_params is None:
    param_key = 'control'
    param_range = [None]
else:
    param_key = experiment_params[0]
    param_range = experiment_params[1]

In [ ]:
# Quartiles Table
result_key = "total_time"
columns = [result_key, f'25% {result_key}', f'50% {result_key}', f'75% {result_key}', f'std {result_key}']
quartiles = pd.DataFrame(columns=columns)

for i, param in enumerate(param_range):
    values = reader[result_key]
    if param_key != 'control':
        values = values[reader[param_key] == param]
    quartiles.loc[i] = [
        result_key, 
        *values.quantile([.25, .5, .75]),
        values.std(),
    ]
display(HTML(quartiles.to_html()))
del values, quartiles